In [1]:
from core import Compiler

In [2]:
drizzle_schema = """
import { integer, pgTable, text } from "drizzle-orm/pg-core";

export const gymEquipmentTable = pgTable("gym_equipment", {
    id: integer().primaryKey().generatedAlwaysAsIdentity(),
    name: text(),
    description: text(),
    price: integer(),
    stock: integer(),
});
""".strip()

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")

In [4]:
result = compiler.compile_drizzle(drizzle_schema)

In [ ]:
result

In [ ]:
print(result["stdout"])

In [7]:
typespec_schema = """
import "./helpers.js";

extern dec llm_func(target: unknown, history: valueof int32);

//{{typespec_definitions}}
""".strip()

In [8]:
result = compiler.compile_typespec(typespec_schema)

In [ ]:
result

## Test Server

In [10]:
import requests

In [11]:
ADDR = "http://localhost:8080" # assuming runs with HOST=0.0.0.0 PORT=8080

In [12]:
drizzle_result = requests.post(
    f"{ADDR}/compile/drizzle",
    json={"payload": drizzle_schema},
)

In [ ]:
drizzle_result.json()

In [14]:
typespec_result = requests.post(
    f"{ADDR}/compile/typespec",
    json={"payload": typespec_schema},
)

In [ ]:
typespec_result.json()